In [1]:
import pandas as pd 
import os
import sys
import pandas as pd
import numpy as np
from torch.utils.data import *
from tqdm import tqdm

sys.path.append('../src/')
sys.path.append('..')

from src.models.lib.neural import GeneClassifier

In [2]:
from src.models.lib.data import *
from src.helper import *

In [3]:
test = GeneExpressionData(
    filename='../data/interim/allen_cortex_T.csv',
    labelname='../data/processed/labels/allen_cortex_labels.csv',
    class_label='Type',
    cast=True,
    skip=3,
)

test.__getitem__(0)

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), 7)

In [4]:
test[3]

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), 7)

In [5]:
# test.columns

In [6]:
%%time

refgenes = gene_intersection()

CPU times: user 2.28 s, sys: 119 ms, total: 2.4 s
Wall time: 2.47 s


In [7]:
from torch.utils.data import DataLoader, ConcatDataset

loader = DataLoader(test, batch_size=4)
sample = next(iter(loader))
sample = sample[0].numpy()

In [8]:
def clean_sample(sample, refgenes, currgenes):
    intersection = np.intersect1d(currgenes, refgenes, return_indices=True)
    indices = intersection[1] # List of indices in currgenes that equal refgenes 
    
    axis = (1 if sample.ndim == 2 else 0)
    sample = np.sort(sample, axis=axis)
    sample = np.take(sample, indices, axis=axis)

    return torch.from_numpy(sample)

In [9]:
datafiles, labelfiles = list(INTERIM_DATA_AND_LABEL_FILES_LIST.keys()), list(INTERIM_DATA_AND_LABEL_FILES_LIST.values())

datafiles = [os.path.join('..', 'data', 'interim', f) for f in datafiles]
labelfiles = [os.path.join('..', 'data', 'processed/labels', f) for f in labelfiles]
datafiles, labelfiles

(['../data/interim/primary_bhaduri_T.csv',
  '../data/interim/allen_cortex_T.csv',
  '../data/interim/allen_m1_region_T.csv',
  '../data/interim/whole_brain_bhaduri_T.csv'],
 ['../data/processed/labels/primary_bhaduri_labels.csv',
  '../data/processed/labels/allen_cortex_labels.csv',
  '../data/processed/labels/allen_m1_region_labels.csv',
  '../data/processed/labels/whole_brain_bhaduri_labels.csv'])

In [10]:
train = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
loader = DataLoader(train, batch_size=4)
currgenes = train.columns

In [11]:
onedsample = train[0][0]
len(onedsample)

19765

In [12]:
t = (clean_sample(onedsample, refgenes, currgenes))
t

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [13]:
len(t)

16604

In [14]:
# twodsample = next(iter(loader))[0]
# twodsample

In [15]:
# %%timeit

# clean_sample(twodsample, refgenes, currgenes)

In [16]:
# for X, y in tqdm(loader):
#     X = clean_sample(X, refgenes, currgenes)

In [17]:
sample.ndim

2

In [18]:
temp = pd.read_csv(datafiles[0], nrows=1, header=1).columns 

In [19]:
# cols = []
# for file in datafiles:
#     # Read in columns, split by | (since some are PVALB|PVALB), and make sure all are uppercase
#     temp = pd.read_csv(file, nrows=1, header=1).columns 
#     temp = [x.split('|')[0].upper().strip() for x in temp]
    
#     print(f'Temp is {temp[0:5]}...')
#     cols.append(set(temp))

# unique = list(set.intersection(*cols))
# unique = sorted(unique)

In [20]:
# len(unique)

In [21]:
# temp = pd.read_csv(datafiles[0], nrows=1, header=1).columns 
# temp = [x.strip().upper() for x in temp]
# l = train.features

In [22]:
# l == temp

In [23]:
# len(set(unique).intersection(l))

In [24]:
# len(set(unique))

In [25]:
# len(set(unique).intersection([x.upper().strip() for x in l]))

In [26]:
train = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
loader = DataLoader(train, batch_size=4)

model = GeneClassifier(
    N_features=len(train.columns),
    N_labels=len(train.labels)
)

Model initialized. N_features = 19765, N_labels = 9. Metrics are {'accuracy': <function accuracy at 0x7fb47d8b5f70>, 'precision': <function precision at 0x7fb47d8d65e0>, 'recall': <function recall at 0x7fb47d8d6700>} and weighted_metrics = False


In [27]:
sample = next(iter(loader))[0]
sample

tensor([[2.8467, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.8507, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.6067, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [28]:
# %%timeit

# model(sample)

Now let's time iterating over our dataloader with and without the extra data cleaning

In [29]:
# for X, y in tqdm(loader):
#     X
#     model(X)

In [30]:
train = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
loader = DataLoader(train, batch_size=4)

model = GeneClassifier(
    N_features=len(refgenes),
    N_labels=len(train.labels)
)

# for X, y in tqdm(loader):
#     X = clean_sample(X, refgenes, train.columns)
#     model(X)

Model initialized. N_features = 16604, N_labels = 9. Metrics are {'accuracy': <function accuracy at 0x7fb47d8b5f70>, 'precision': <function precision at 0x7fb47d8d65e0>, 'recall': <function recall at 0x7fb47d8d6700>} and weighted_metrics = False


In [31]:
loader.dataset

In [32]:
train, test, insize, numlabels, weights = generate_datasets(datafiles, labelfiles, 'Type')

In [33]:
train.datasets[0].name

'../data/interim/primary_bhaduri_T.csv'

In [34]:
datafiles

['../data/interim/primary_bhaduri_T.csv',
 '../data/interim/allen_cortex_T.csv',
 '../data/interim/allen_m1_region_T.csv',
 '../data/interim/whole_brain_bhaduri_T.csv']

In [35]:
[train.datasets[i].name for i in range(len(datafiles))]

['../data/interim/primary_bhaduri_T.csv',
 '../data/interim/allen_cortex_T.csv',
 '../data/interim/allen_m1_region_T.csv',
 '../data/interim/whole_brain_bhaduri_T.csv']

We can see that it's much faster to clean the sample on each minibatch, since numpy clearly scales well under-the-hood. Therefore, we'll have to write a manual training loop as we can no longer use pytorch lightning.

In [36]:
from pytorch_lightning import Trainer

# train = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
# loader = DataLoader(train, batch_size=4)

# model = GeneClassifier(
#     N_features=len(train.columns),
#     load
# )

In [37]:
combined, test, insize, numlabels, weights = generate_datasets(datafiles, labelfiles, 'Type')
numlabels

18

In [38]:
train = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
trainloader = DataLoader(train, batch_size=4)

net = GeneClassifier(
    N_features=len(train.columns),
    N_labels=max(train.labels)
)

Model initialized. N_features = 19765, N_labels = 17. Metrics are {'accuracy': <function accuracy at 0x7fb47d8b5f70>, 'precision': <function precision at 0x7fb47d8d65e0>, 'recall': <function recall at 0x7fb47d8d6700>} and weighted_metrics = False


In [39]:
loaders = []
refgenes = gene_intersection()

In [40]:
for datafile, labelfile in zip(datafiles, labelfiles):
    data = GeneExpressionData(
            datafile,
            labelfile,
            'Type',
            cast=False,
    )
    
#     print(data[0][0][0:5])
    loaders.append(data)

In [41]:
# for data in loaders:
#     print(data.name)
#     print(data[0][0][0:5])

In [42]:
loaders = [DataLoader(data, batch_size=4) for data in loaders]

In [80]:
df1 = pd.read_csv(datafiles[0], nrows=50, header=1)
df2 = pd.read_csv(datafiles[1], nrows=50, header=1)
df3 = pd.read_csv(datafiles[2], nrows=50, header=1)
df4 = pd.read_csv(datafiles[3], nrows=50, header=1)

df1_labels = pd.read_csv(labelfiles[0])
df2_labels = pd.read_csv(labelfiles[1])
df3_labels = pd.read_csv(labelfiles[2])
df4_labels = pd.read_csv(labelfiles[3])

In [81]:
df1

,FO538757.2,AP006222.2,RP11-206L10.9,FAM41C,NOC2L,AGRN,C1orf159,ACAP3,CPSF3L,AURKAIP1,...,SDC1,HS1BP3,RP11-130L8.1,KLHL29,RP11-557N21.1,MFSD2B,FAM228A,ITSN2,AC009228.1,ADCY3
0,2.846667,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,1.850696,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.606674,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,3.506125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,3.125501,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,2.453861,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2.590280,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,2.187795,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
df2

,3.8-1.2,3.8-1.3,3.8-1.4,3.8-1.5,5-HT3C2,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,...,HIST1H2AG,HIST1H2AH,HIST1H2AI,HIST1H2AJ,HIST1H2AK,HIST1H2AL,HIST1H2AM,HIST1H2APS1,HIST1H2APS2,HIST1H2APS3
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,118,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,159,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
df3

,DDX11L1|DDX11L1,WASH7P|WASH7P,MIR6859-1|MIR6859-1,MIR1302-2|MIR1302-2,FAM138A|FAM138A,LOC105379212|LOC105379212,OR4G4P|OR4G4P,OR4G11P|OR4G11P,OR4F5|OR4F5,LOC105379213|LOC105379213,...,LOC105374091|LOC105374091,PLXNA1|PLXNA1,LOC105374092|LOC105374092,C3orf56|C3orf56,LOC101060159|LOC101060159,LOC105374093|LOC105374093,LOC100419008|LOC100419008,LOC101927123|LOC101927123,LOC105374094|LOC105374094,LINC01471|LINC01471
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
df4

,RP11-34P13.7|RP11-34P13.7,FO538757.2|FO538757.2,AP006222.2|AP006222.2,RP4-669L17.10|RP4-669L17.10,RP11-206L10.9|RP11-206L10.9,LINC00115|LINC00115,FAM41C|FAM41C,RP11-54O7.16|RP11-54O7.16,RP11-54O7.1|RP11-54O7.1,RP11-54O7.2|RP11-54O7.2,...,FADS2|FADS2,FADS1|FADS1,FADS3|FADS3,BEST1|BEST1,FTH1|FTH1,RP11-810P12.5|RP11-810P12.5,INCENP|INCENP,RP11-703H8.7|RP11-703H8.7,ASRGL1|ASRGL1,CTD-2531D15.5|CTD-2531D15.5
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
df1_data = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
df2_data = GeneExpressionData(datafiles[1], labelfiles[1], 'Type', skip=2)
df3_data = GeneExpressionData(datafiles[2], labelfiles[2], 'Type', skip=3)
df4_data = GeneExpressionData(datafiles[3], labelfiles[3], 'Type', skip=3)

In [105]:
datasets = [df1_data, df2_data, df3_data, df4_data]

In [106]:
for dataset in datasets:
    print(dataset.columns[0:10])

['FO538757.2', 'AP006222.2', 'RP11-206L10.9', 'FAM41C', 'NOC2L', 'AGRN', 'C1ORF159', 'ACAP3', 'CPSF3L', 'AURKAIP1']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['DDX11L1', 'WASH7P', 'MIR6859-1', 'MIR1302-2', 'FAM138A', 'LOC105379212', 'OR4G4P', 'OR4G11P', 'OR4F5', 'LOC105379213']
['RP11-34P13.7', 'FO538757.2', 'AP006222.2', 'RP4-669L17.10', 'RP11-206L10.9', 'LINC00115', 'FAM41C', 'RP11-54O7.16', 'RP11-54O7.1', 'RP11-54O7.2']


In [116]:
df2_data = GeneExpressionData(datafiles[1], labelfiles[1], 'Type', skip=2)
len(df2_data[0][0])

50281

In [120]:
df2_data[0]

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), 7)

In [89]:
all(np.isclose(df1_data[0][0], df1.loc[0, :].values))

True

In [113]:
all(np.isclose(df2_data[0][0], df2.loc[0, :].values))

False

In [91]:
all(np.isclose(df3_data[0][0], df3.loc[0, :].values))

True

In [ ]:
all(np.isclose(df3_data[0][0], df3.loc[0, :].values))

The falses makes sense since the indices arent being changed when we're reading in the pure dataframes

In [82]:
for i in range(50):
    print(all(np.isclose(df1_data[i][0], df1.loc[i, :].values)))
    print(all(np.isclose(df2_data[i][0], df2.loc[i, :].values)))
    print(all(np.isclose(df3_data[i][0], df3.loc[i, :].values)))
    print(all(np.isclose(df3_data[i][0], df3.loc[i, :].values)))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
T

In [83]:
df1_data = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
df2_data = GeneExpressionData(datafiles[1], labelfiles[1], 'Type', skip=2)
df3_data = GeneExpressionData(datafiles[2], labelfiles[2], 'Type', skip=3)
df4_data = GeneExpressionData(datafiles[3], labelfiles[3], 'Type', skip=3)

In [ ]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(100):  # loop over the dataset multiple times
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')